# The Price is Right

 a ScanningAgent that looks for promising deals by subscribing to RSS feeds.

In [10]:
# Add project root so we can import agents from References/
import sys
from pathlib import Path
_root = Path.cwd() if (Path.cwd() / "agents").is_dir() else Path.cwd().parent
if str(_root) not in sys.path:
    sys.path.insert(0, str(_root))

In [11]:
import os
from dotenv import load_dotenv
from openai import OpenAI
from agents.deals import ScrapedDeal, DealSelection
import logging
import requests
load_dotenv(override=True)
openai = OpenAI()
MODEL = 'gpt-5-mini'

In [12]:
deals = ScrapedDeal.fetch(show_progress=True)

100%|██████████| 3/3 [00:29<00:00,  9.90s/it]


In [13]:
len(deals)

30

In [14]:
deals[10].describe()

'Title: Straight Talk Prepaid Home Internet All-in-One Router for $47 + free shipping\nDetails: Use promo code "EBAYVIP2026SALE" for this price, a $23 low. Buy Now at eBay\nFeatures: \nURL: https://www.dealnews.com/products/Straight-Talk-Wireless/Prepaid-Home-Internet-All-in-One-Router/497665.html?iref=rss-c39'

### We are going to ask GPT-5-mini to summarize deals and identify their price

In [15]:
SYSTEM_PROMPT = """You identify and summarize the 5 most detailed deals from a list, by selecting deals that have the most detailed, high quality description and the most clear price.
Respond strictly in JSON with no explanation, using this format. You should provide the price as a number derived from the description. If the price of a deal isn't clear, do not include that deal in your response.
Most important is that you respond with the 5 deals that have the most detailed product description with price. It's not important to mention the terms of the deal; most important is a thorough description of the product.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 
"""

USER_PROMPT_PREFIX = """Respond with the most promising 5 deals from this list, selecting those which have the most detailed, high quality product description and a clear price that is greater than 0.
You should rephrase the description to be a summary of the product itself, not the terms of the deal.
Remember to respond with a short paragraph of text in the product_description field for each of the 5 items that you select.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

Deals:

"""

USER_PROMPT_SUFFIX = "\n\nInclude exactly 5 deals, no more."

In [16]:
# this makes a suitable user prompt given scraped deals

def make_user_prompt(scraped):
    user_prompt = USER_PROMPT_PREFIX
    user_prompt += '\n\n'.join([scrape.describe() for scrape in scraped])
    user_prompt += USER_PROMPT_SUFFIX
    return user_prompt

In [17]:
# Let's create a user prompt for the deals we just scraped, and look at how it begins

user_prompt = make_user_prompt(deals)
print(user_prompt[:2000])
messages = [{"role": "system", "content": SYSTEM_PROMPT}, {"role": "user", "content": user_prompt}]

Respond with the most promising 5 deals from this list, selecting those which have the most detailed, high quality product description and a clear price that is greater than 0.
You should rephrase the description to be a summary of the product itself, not the terms of the deal.
Remember to respond with a short paragraph of text in the product_description field for each of the 5 items that you select.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

Deals:

Title: onn. Reusable 35MM Camera w/ Flash for $5 + free shipping
Details: Use promo code "EBAYVIP2026SALE" for this price, an $11 low. Buy Now at eBay
Features: 
URL: https://www.dealnews.com/onn-Reusable-35-MM-Camera-w-Flash-for-5-free-shipping/21810349.html?iref=rss-c142

Title: Certified Refurb EcoFlow Delta Pro 3 4,000Wh Power Station for $1,559 + free shipping
Details: Use promo 

In [19]:
response = openai.chat.completions.parse(model=MODEL, messages=messages, response_format=DealSelection, reasoning_effort="minimal")
results = response.choices[0].message.parsed
results

DealSelection(deals=[Deal(product_description='Certified-refurbished EcoFlow Delta Pro 3 is a large-capacity portable power station with a 4,000 Wh battery designed for multi-day home backup and off-grid power. It supports high-wattage appliance loads, multiple AC outlets, and likely offers smart app monitoring, expandable battery capability, and fast recharge options via AC, solar, or EV charging. The unit is sold refurbished and includes a 2-year Allstate warranty.', price=1559.0, url='https://www.dealnews.com/Certified-Refurb-Eco-Flow-Delta-Pro-3-4-000-Wh-Power-Station-for-1-559-free-shipping/21810336.html?iref=rss-c142'), Deal(product_description='Open-box Apple AirPods 4 are the latest AirPods model with a smaller charging case, USB‑C charging, and up to 30 hours of combined battery life. They include the same core wireless audio features Apple offers—seamless pairing with Apple devices, spatial audio support, and an included trial of Apple Music—and are sold new but without origi

In [20]:
for deal in results.deals:
    print(deal.product_description)
    print(deal.price)
    print(deal.url)
    print()


Certified-refurbished EcoFlow Delta Pro 3 is a large-capacity portable power station with a 4,000 Wh battery designed for multi-day home backup and off-grid power. It supports high-wattage appliance loads, multiple AC outlets, and likely offers smart app monitoring, expandable battery capability, and fast recharge options via AC, solar, or EV charging. The unit is sold refurbished and includes a 2-year Allstate warranty.
1559.0
https://www.dealnews.com/Certified-Refurb-Eco-Flow-Delta-Pro-3-4-000-Wh-Power-Station-for-1-559-free-shipping/21810336.html?iref=rss-c142

Open-box Apple AirPods 4 are the latest AirPods model with a smaller charging case, USB‑C charging, and up to 30 hours of combined battery life. They include the same core wireless audio features Apple offers—seamless pairing with Apple devices, spatial audio support, and an included trial of Apple Music—and are sold new but without original retail packaging.
60.0
https://www.dealnews.com/products/Apple/Apple-Air-Pods-4/48178

In [21]:
root = logging.getLogger()
root.setLevel(logging.INFO)

In [23]:
from agents.scanner_agent import ScannerAgent

In [24]:
agent = ScannerAgent()
result = agent.scan()

INFO:root:[Scanner Agent] Scanner Agent is initializing
INFO:root:[Scanner Agent] Scanner Agent is ready
INFO:root:[Scanner Agent] Scanner Agent is about to fetch deals from RSS feed
INFO:root:[Scanner Agent] Scanner Agent received 30 deals not already scraped
INFO:root:[Scanner Agent] Scanner Agent is calling OpenAI using Structured Outputs
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:[Scanner Agent] Scanner Agent received 5 selected deals with price>0 from OpenAI


In [25]:
result

DealSelection(deals=[Deal(product_description='Certified refurbished EcoFlow Delta Pro 3 is a large-capacity portable power station providing 4,000 watt-hours of usable battery capacity for whole-home backup or extended off-grid use. It supports high-output AC charging, multiple USB and DC outputs, and can integrate with solar arrays and additional battery modules for scalable energy storage. The unit includes a certified refurb inspection and a 2-year Allstate warranty for added assurance.', price=1559.0, url='https://www.dealnews.com/Certified-Refurb-Eco-Flow-Delta-Pro-3-4-000-Wh-Power-Station-for-1-559-free-shipping/21810336.html?iref=rss-c142'), Deal(product_description='Open-box Apple AirPods 4 are the latest-generation true wireless earbuds featuring a compact USB-C charging case and up to 30 hours of combined battery life. They offer spatial audio and seamless integration with Apple devices, include three months of Apple Music, and come without original retail packaging but othe

### Introducing Pushover

Pushover is a nifty tool for sending Push Notifications to your phone.

It's super easy to set up and install!

Simply visit https://pushover.net/ and click 'Login or Signup' on the top right to sign up for a free account, and create your API keys.

Once you've signed up, on the home screen, click "Create an Application/API Token", and give it any name (like AIEngineer) and click Create Application.

Then add 2 lines to your `.env` file:

PUSHOVER_USER=_put the key that's on the top right of your Pushover home screen and probably starts with a u_  
PUSHOVER_TOKEN=_put the key when you click into your new application called Agents (or whatever) and probably starts with an a_

Remember to save your `.env` file, and run `load_dotenv(override=True)` after saving, to set your environment variables.

Finally, click "Add Phone, Tablet or Desktop" to install on your phone.

In [ ]:
load_dotenv(override=True)

In [ ]:
pushover_user = os.getenv('PUSHOVER_USER')
pushover_token = os.getenv('PUSHOVER_TOKEN')
pushover_url = "https://api.pushover.net/1/messages.json"

In [ ]:
if pushover_user:
    print(f"Pushover user found and starts with {pushover_user[0]}")
else:
    print("Pushover user not found")

if pushover_token:
    print(f"Pushover token found and starts with {pushover_token[0]}")
else:
    print("Pushover token not found")

In [ ]:
def push(message):
    print(f"Push: {message}")
    payload = {"user": pushover_user, "token": pushover_token, "message": message}
    requests.post(pushover_url, data=payload)

In [ ]:
push("MASSIVE DEAL!!")

In [ ]:
from agents.messaging_agent import MessagingAgent

agent = MessagingAgent()
agent.push("SUCH A MASSIVE DEAL!!")

In [ ]:
agent.notify("A special deal on Sumsung 60 inch LED TV going at a great bargain", 300, 1000, "www.samsung.com")